In [1]:
import os
import gensim
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

In [2]:
gensim.__path__[0]


'/Users/ariedamuco/opt/anaconda3/lib/python3.8/site-packages/gensim'

In [3]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [4]:
len(train_corpus)
#test_corpus

300

In [5]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)


In [6]:
a = model.build_vocab(train_corpus)

In [7]:
type(a)

NoneType

In [8]:
dir(a)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [9]:
print(f"Word 'australia' appeared {model.wv.get_vecattr('australia', 'count')} times in the training corpus.")

Word 'australia' appeared 157 times in the training corpus.


In [10]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [11]:
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-0.1298413  -0.26351744 -0.16206652  0.25780457 -0.12477654 -0.02890203
  0.0207357  -0.06182202 -0.33271995 -0.26591545  0.05074275  0.03833982
 -0.01972873  0.04386122 -0.12174339 -0.03690551  0.11654571  0.27582285
  0.12707412 -0.15202868  0.07332347 -0.04773286  0.27094755 -0.04652206
  0.06794368 -0.02625057 -0.25737736  0.00850258 -0.2483172  -0.04580629
  0.45191035  0.01146388  0.17048824  0.0722725   0.18929929  0.14721116
 -0.04898207 -0.25986546  0.01616161 -0.03928765  0.07648917  0.04194674
 -0.04888457 -0.1670667   0.27372187  0.1297825  -0.06701881 -0.106051
  0.24347265 -0.08975735]


In [12]:
sentence = ['only', 'you', 'can', 'prevent', 'forest', 'fires']
vec = 0
for element in sentence:
    element = model.infer_vector([element])
    vec = vec + element
    
vec=vec/len(sentence)
        
    

In [13]:
vec

array([-0.23950106, -0.02897451, -0.06777478, -0.00580347, -0.04478599,
       -0.09598824,  0.04626723,  0.07935983, -0.15244988, -0.106529  ,
        0.08025175, -0.10874251,  0.0317119 , -0.04808137, -0.12334748,
       -0.02115649,  0.14514613,  0.07758651, -0.06570814, -0.10085595,
        0.11167425,  0.05832593,  0.24622466, -0.10549525,  0.09983471,
       -0.05586858, -0.16435085, -0.02864029, -0.13045241, -0.13050914,
        0.10650271,  0.02406158, -0.00510718,  0.16439267, -0.02118719,
        0.06664506,  0.00870092, -0.10596772,  0.03006651, -0.00634379,
        0.11851584,  0.06964938, -0.08974657, -0.05687732,  0.21982867,
        0.12488998,  0.0994859 , -0.09217793,  0.11379348,  0.02167792],
      dtype=float32)

In [14]:
len(model.infer_vector(train_corpus[1].words))

50

In [15]:
ranks = []
second_ranks = []
first_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    first_ranks.append(sims[0])
    second_ranks.append(sims[1])

In [16]:
first_ranks

[(0, 0.9730887413024902),
 (1, 0.9550246596336365),
 (2, 0.9374590516090393),
 (3, 0.9116206765174866),
 (4, 0.9186385869979858),
 (5, 0.9173019528388977),
 (6, 0.9711156487464905),
 (7, 0.9440189599990845),
 (8, 0.9706077575683594),
 (9, 0.9526470303535461),
 (10, 0.9443311095237732),
 (11, 0.9186559915542603),
 (12, 0.9505726099014282),
 (13, 0.9423649311065674),
 (14, 0.9652225375175476),
 (15, 0.9314064383506775),
 (16, 0.948931872844696),
 (17, 0.9565064311027527),
 (18, 0.9090035557746887),
 (19, 0.9549897313117981),
 (20, 0.9063193202018738),
 (21, 0.958569347858429),
 (22, 0.9407030344009399),
 (23, 0.9186989068984985),
 (24, 0.9182595014572144),
 (25, 0.9511631727218628),
 (26, 0.9174779653549194),
 (27, 0.9581631422042847),
 (28, 0.906255304813385),
 (29, 0.9377850294113159),
 (30, 0.9034258127212524),
 (31, 0.9518626928329468),
 (32, 0.9369925260543823),
 (33, 0.9675165414810181),
 (34, 0.9684593081474304),
 (35, 0.9636623859405518),
 (36, 0.9484882354736328),
 (37, 0.972077

In [17]:
second_ranks

[(48, 0.9009266495704651),
 (143, 0.7628950476646423),
 (21, 0.8521811366081238),
 (269, 0.6855833530426025),
 (33, 0.7279812693595886),
 (218, 0.6723355650901794),
 (17, 0.8034862875938416),
 (15, 0.8375425934791565),
 (48, 0.8977537751197815),
 (8, 0.8198491930961609),
 (264, 0.7979761958122253),
 (188, 0.7233511805534363),
 (26, 0.7131016850471497),
 (292, 0.7325883507728577),
 (277, 0.8672385811805725),
 (27, 0.7963988184928894),
 (139, 0.8452092409133911),
 (6, 0.838197648525238),
 (71, 0.7155660390853882),
 (40, 0.7907366752624512),
 (156, 0.6495237946510315),
 (43, 0.875139594078064),
 (179, 0.7793442010879517),
 (56, 0.7290645837783813),
 (188, 0.6434575319290161),
 (10, 0.8611453175544739),
 (12, 0.7461656928062439),
 (15, 0.8812684416770935),
 (56, 0.7032357454299927),
 (173, 0.7514073252677917),
 (121, 0.7149563431739807),
 (251, 0.7335489392280579),
 (39, 0.7664366364479065),
 (8, 0.879328191280365),
 (12, 0.6737383604049683),
 (127, 0.7426714301109314),
 (224, 0.5261580348

In [18]:
train_corpus[4]

TaggedDocument(words=['six', 'midwives', 'have', 'been', 'suspended', 'at', 'wollongong', 'hospital', 'south', 'of', 'sydney', 'for', 'inappropriate', 'use', 'of', 'nitrous', 'oxide', 'during', 'work', 'hours', 'on', 'some', 'occasions', 'while', 'women', 'were', 'in', 'labour', 'the', 'illawarra', 'area', 'health', 'service', 'says', 'that', 'following', 'an', 'investigation', 'of', 'unprofessional', 'conduct', 'further', 'four', 'midwives', 'have', 'been', 'relocated', 'to', 'other', 'areas', 'within', 'the', 'hospital', 'the', 'service', 'chief', 'executive', 'officer', 'tony', 'sherbon', 'says', 'no', 'one', 'was', 'put', 'at', 'risk', 'because', 'other', 'staff', 'not', 'involved', 'in', 'the', 'use', 'of', 'nitrous', 'oxide', 'were', 'able', 'to', 'take', 'over', 'caring', 'for', 'women', 'in', 'labour', 'well', 'we', 're', 'very', 'concerned', 'and', 'the', 'body', 'of', 'midwives', 'to', 'the', 'hospital', 'there', 'are', 'over', 'midwives', 'that', 'work', 'in', 'our', 'servic

In [19]:
import collections

counter = collections.Counter(ranks)
print(counter)

Counter({0: 292, 1: 8})


In [20]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [21]:
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)


SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):



In [22]:
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))


MOST (299, 0.9493477940559387): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with h

In [23]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = first_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (247): «the royal commission into hih has been adjourned until monday after interviewing of the first witness ended abruptly lawyers acting on behalf of several former hih directors requested that cross examination of david lombe and an inspector appointed by insurance watchdog the australian prudential regulation authority apra be held at later date the commission was due to hear evidence from mr lombe after the details of his report for apra were read in this morning hearing the report raises allegations that corporate governance was inadequate and concealed the true state of hih financial position the report questions the solvency of hih prior to it being placed into provisional liquidation on march it raises issues of intangible assets like tax and goodwill being treated as tangible for the acquisition of allianz to boost the financial position of hih the report also says significant losses by hih in the united states and the united kingdom were concealed from apra a

In [24]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (22): «the biowarfare expert under scrutiny in the anthrax attacks declared am not the anthrax killer and lashed out today against attorney general john ashcroft for calling him person of interest in the investigation for the second time in two weeks the scientist went before throng of reporters outside his lawyer office to profess his innocence and decry the attention from law enforcers that he contends has destroyed his life»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (23, 0.6958336234092712): «americans fears about airplane security continue to increase after man made it through two separate flights with loaded gun in his carry on luggage the man was finally stopped before boarding third plane in memphis the man had travelled from florida to atlanta and then atlanta to memphis he was attempting to board his return flight last night when he was stopped by security personnel for random check they discovered loaded mm beretta semi auto

### Challenge: Use the senator speeches in the folder 105-extracted-date and use doc2vec to find whose senator speech is closest to senator Biden. Use sen105kh_fix.csv and/or Wikipedia to validate your findings (i.e., understand if the most similar speeches are senators from the same state and/party).  Describe your findings. Compare with the outcome you got/will get using cosine similarity.

In [25]:
#https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html